# LAGO (Linear Approximation for Graph Optimization)

The `gtsam::lago` namespace provides functions for initializing `Pose2` estimates in a 2D SLAM factor graph.
LAGO stands for Linear Approximation for Graph Optimization. It leverages the structure of typical 2D SLAM problems to efficiently compute an initial guess, particularly for the orientations, which are often the most challenging part for nonlinear solvers.

The core idea is:
1.  **Estimate Orientations:** Assume orientations are independent of translations and solve a linear system just for the orientations ($	heta$). This exploits the fact that the orientation part of the `Pose2` `BetweenFactor` error is approximately linear for small errors.
2.  **Estimate Translations:** Given the estimated orientations, compute the translations by solving another linear system.

Key functions:
*   `initializeOrientations(graph)`: Computes initial estimates for the `Rot2` (orientation) components of the poses in the graph.
*   `initialize(graph)`: Computes initial estimates for the full `Pose2` variables (orientations and translations).
*   `initialize(graph, initialGuess)`: Corrects only the orientation part of a given `initialGuess` using LAGO.

LAGO typically assumes the graph contains a spanning tree of odometry measurements and a prior on the starting pose.

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/slam/doc/lago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet gtsam

In [2]:
import gtsam
import gtsam.utils.plot as gtsam_plot
import matplotlib.pyplot as plt
import numpy as np
from gtsam import NonlinearFactorGraph, Values, Pose2, Point3, PriorFactorPose2, BetweenFactorPose2
from gtsam import lago
from gtsam import symbol_shorthand

X = symbol_shorthand.X

## Example Initialization

We'll create a simple 2D pose graph with odometry and a loop closure, then use `lago.initialize`.

In [3]:
# 1. Create a NonlinearFactorGraph with Pose2 factors
graph = NonlinearFactorGraph()

# Add a prior on the first pose
prior_mean = Pose2(0.0, 0.0, 0.0)
prior_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.1, 0.1, 0.05]))
graph.add(PriorFactorPose2(X(0), prior_mean, prior_noise))

# Add odometry factors (simulating moving in a square)
odometry_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
graph.add(BetweenFactorPose2(X(0), X(1), Pose2(2.0, 0.0, 0.0), odometry_noise))
graph.add(BetweenFactorPose2(X(1), X(2), Pose2(2.0, 0.0, np.pi/2), odometry_noise))
graph.add(BetweenFactorPose2(X(2), X(3), Pose2(2.0, 0.0, np.pi/2), odometry_noise))
graph.add(BetweenFactorPose2(X(3), X(4), Pose2(2.0, 0.0, np.pi/2), odometry_noise))

# Add a loop closure factor
loop_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.25, 0.25, 0.15]))
# Ideal loop closure would be Pose2(2.0, 0.0, np.pi/2)
measured_loop = Pose2(2.1, 0.1, np.pi/2 + 0.05)
graph.add(BetweenFactorPose2(X(4), X(0), measured_loop, loop_noise))

graph.print("Original Factor Graph:\n")

# 2. Perform LAGO initialization
initial_estimate_lago = lago.initialize(graph)

print("\nInitial Estimate from LAGO:\n")
initial_estimate_lago.print()

# 3. Visualize the LAGO estimate (optional)
plt.figure(1)
for key in initial_estimate_lago.keys():
    gtsam_plot.plot_pose2(1, initial_estimate_lago.atPose2(key), 0.5)
plt.title("LAGO Initial Estimate")
plt.axis('equal')
# plt.show()
plt.close() # Close plot to prevent display in output

# 4. Compare orientation initialization
initial_orientations = lago.initializeOrientations(graph)
print("\nLAGO Orientations (VectorValues):")
initial_orientations.print()

print("Orientations from full LAGO Values:")
for i in range(5):
    print(f"  X({i}): {initial_estimate_lago.atPose2(X(i)).theta():.4f}")

Original Factor Graph:
size: 6

Factor 0: PriorFactor on x0
  prior mean:  (0, 0, 0)
  noise model: diagonal sigmas [0.1; 0.1; 0.05];

Factor 1: BetweenFactor(x0,x1)
  measured:  (2, 0, 0)
  noise model: diagonal sigmas [0.2; 0.2; 0.1];

Factor 2: BetweenFactor(x1,x2)
  measured:  (2, 0, 1.57079633)
  noise model: diagonal sigmas [0.2; 0.2; 0.1];

Factor 3: BetweenFactor(x2,x3)
  measured:  (2, 0, 1.57079633)
  noise model: diagonal sigmas [0.2; 0.2; 0.1];



Factor 4: BetweenFactor(x3,x4)
  measured:  (2, 0, 1.57079633)
  noise model: diagonal sigmas [0.2; 0.2; 0.1];

Factor 5: BetweenFactor(x4,x0)
  measured:  (2.1, 0.1, 1.62079633)
  noise model: diagonal sigmas [0.25; 0.25; 0.15];



IndexError: invalid map<K, T> key

## Notes

- LAGO provides a good initial guess, especially for orientations, which can significantly help the convergence of nonlinear optimizers.
- It assumes the graph structure allows for the orientation estimation (typically requires a spanning tree and a prior).
- The translation estimates are computed based on the fixed, estimated orientations.